In [1]:
%cd ..

/home/dawood/Workspace/compoundGAN


In [2]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from data.zinc import ZINC250K, ZINC_collate
from models.gcn import GCN

In [3]:
model = GCN(59, 13, 256, [64, 128, 128])

In [4]:
dataset = ZINC250K('/home/dawood/Datasets/ZINC250K/data.csv')

In [7]:
dataloader = DataLoader(dataset, 64, shuffle=True, collate_fn=ZINC_collate, num_workers=8)

In [8]:
num_nodes = []
for i, g in tqdm(enumerate(dataloader)):
    #num_nodes.extend(g.batch_num_nodes)
    break


0it [00:00, ?it/s]
1it [00:02,  2.61s/it]
9it [00:03,  1.87s/it]
10it [00:04,  1.48s/it]
17it [00:05,  1.07s/it]
18it [00:05,  1.11it/s]
24it [00:06,  1.47it/s]
26it [00:06,  1.96it/s]
30it [00:07,  2.63it/s]
32it [00:07,  2.61it/s]
34it [00:08,  2.55it/s]
38it [00:09,  2.96it/s]
42it [00:10,  3.22it/s]
46it [00:11,  3.70it/s]
50it [00:12,  3.21it/s]
58it [00:14,  3.51it/s]
66it [00:15,  4.07it/s]
74it [00:17,  4.56it/s]
82it [00:18,  4.90it/s]
84it [00:19,  4.27it/s]
88it [00:19,  5.44it/s]
90it [00:20,  4.00it/s]
92it [00:20,  5.03it/s]
96it [00:21,  4.05it/s]
101it [00:22,  5.01it/s]
104it [00:23,  4.44it/s]
108it [00:23,  6.01it/s]
110it [00:23,  6.24it/s]
112it [00:24,  4.23it/s]
113it [00:25,  2.51it/s]
116it [00:25,  3.44it/s]
120it [00:25,  4.28it/s]
122it [00:27,  2.47it/s]
129it [00:28,  2.97it/s]
135it [00:28,  3.88it/s]
136it [00:29,  3.87it/s]
137it [00:29,  3.01it/s]
143it [00:31,  3.25it/s]
147it [00:31,  4.14it/s]
151it [00:33,  3.35it/s]
159it [00:35,  3.46it/s]
167it

KeyboardInterrupt: 

In [10]:
max(num_nodes)

39

In [ ]:
torch.Tensor(atom_y[0]).shape

In [ ]:
mask.shape

In [ ]:
len(atom_y)